In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
import os, math
from collections import Counter

size_mbs = lambda x: int(os.path.getsize(x)
                         / math.pow(1024, 2))

def file_sizes(directory): 
    d = "../data/osm/ny/" + directory
    sizes = [size_mbs(os.path.join(d, f)) \
             for f in os.listdir(d)]
    c = Counter(sizes)
    for size, count in c.items():
        print("{:3d} x {:3d} MB".format(count, size))

In [ ]:
file_sizes("500mb")

In [ ]:
import dask.bag as db
from operator import itemgetter

extract_tag = itemgetter(1, 3)

def top_tags(data):
    return data.filter(
        lambda x: "tag" in x
    ).foldby(
        lambda x: extract_tag(x.split('"')),
        lambda x, _: x + 1, 0
    ).topk(10, itemgetter(1))

In [ ]:
data = db.read_text("../data/osm/ny/500mb/*")
task = top_tags(data)
task

In [ ]:
task.visualize("task1", format="svg");

In [ ]:
task.compute()

In [ ]:
file_sizes("5mb")
data = db.read_text("../data/osm/ny/5mb/*")
task = top_tags(data)
task

In [ ]:
task.compute()

In [ ]:
file_sizes("100mb")
data = db.read_text("../data/osm/ny/100mb/*")
task = top_tags(data)
task

In [ ]:
task.compute()

In [ ]:
data = db.read_text("../data/osm/southamerica/southam*")
task = top_tags(data)
task

In [ ]:
task.compute()